# 导入所需模块

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

---

直方图是图像处理过程中的一种非常重要的分析工具。直方图从图像内部灰度级的角度对图像进行表述，包含十分丰富而重要的信息。从直方图的角度对图像进行处理，可以达到增强图像显示效果的目的。

# 直方图的含义
从统计的角度讲，直方图是图像内灰度值的统计特性与图像灰度值之间的函数，直方图统计图像内各个灰度级出现的次数。从直方图的图形上观察，横坐标是图像中各像素点的灰度级，纵坐标是具有该灰度级（像素值）的像素个数。

有时为了便于表示，也会采用归一化直方图。在归一化直方图中，x轴仍然表示灰度级；y轴不再表示灰度级出现的次数，而是灰度级出现的频率。

在OpenCV的官网上，特别提出了要注意三个概念：DIMS、BINS、RANGE。
- DIMS：表示在绘制直方图时，收集的参数的数量。一般情况下，直方图中收集的数据只有一种，就是灰度级。因此，该值为1。
- RANGE：表示要统计的灰度级范围，一般为[0, 255]。0对应的是黑色，255对应的是白色。
- BINS：参数子集的数目。在处理数据的过程中，有时需要将众多的数据划分为若干个组，再进行分析。

---
# 绘制直方图
Python的模块matplotlib.pyplot中的`hist()`函数能够方便地绘制直方图，我们通常采用该函数直接绘制直方图。除此以外，OpenCV中的`cv2.calcHist()`函数能够计算统计直方图，还可以在此基础上绘制图像的直方图。

## 使用Numpy绘制直方图


In [ ]:
o=cv2.imread("./images/boat.jpg")
cv2.imshow("original",o)
plt.hist(o.ravel(),256)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
o=cv2.imread("./images/boat.bmp")
plt.hist(o.ravel(),16)

## 使用OpenCV绘制直方图
OpenCV提供了函数`cv2.calcHist()`用来计算图像的统计直方图，该函数能统计各个灰度级的像素点个数。利用matplotlib.pyplot模块中的`plot()`函数，可以将函数`cv2.calcHist()`的统计结果绘制成直方图。

### 用cv2.calcHist()函数统计图像直方图信息

In [ ]:
img=cv2.imread("./images/lena.jpg")
hist = cv2.calcHist([img],[0],None,[256],[0,255])
print(type(hist))
print(hist.shape)
print(hist.size)
print(hist)

### plot()函数的使用


In [ ]:
x = [0,1,2,3,4,5,6]
y = [0.3,0.4,2,5,3,4.5,4]
plt.plot(x,y)

In [ ]:
y = [0.3,0.4,2,5,3,4.5,4]
plt.plot(y)

In [ ]:
a = [0.3,0.4,2,5,3,4.5,4]
b=[3,5,1,2,1,5,3]
plt.plot(a,color='r')
plt.plot(b,color='g')

### 绘制统计直方图


In [ ]:
o=cv2.imread("./images/boatGray.bmp")
histb = cv2.calcHist([o],[0],None,[256],[0,255])
plt.plot(histb,color='b')
plt.show()

In [ ]:
o=cv2.imread("./images/girl.bmp")
histb = cv2.calcHist([o],[0],None,[256],[0,255])
histg = cv2.calcHist([o],[1],None,[256],[0,255])
histr = cv2.calcHist([o],[2],None,[256],[0,255])
plt.plot(histb,color='b')
plt.plot(histg,color='g')
plt.plot(histr,color='r')
plt.show()

### 使用掩模绘制直方图
在函数`cv2.calcHist()`中，参数mask用于标识是否使用掩模图像。当使用掩模图像获取直方图时，仅获取掩模参数mask指定区域的直方图。

#### 掩模处理是怎么回事
#### 如何构造掩模图像

In [13]:
mask=np.zeros([600,600],np.uint8)
mask[200:400,200:400]=255
cv2.imshow('mask',mask)
cv2.waitKey()
cv2.destroyAllWindows()

#### 使用掩模绘制直方图

In [ ]:
image=cv2.imread("./images/girl.bmp",cv2.IMREAD_GRAYSCALE)
mask=np.zeros(image.shape,np.uint8)
mask[200:400,200:400]=255
histImage=cv2.calcHist([image],[0],None,[256],[0,255])
histMI=cv2.calcHist([image],[0],mask,[256],[0,255])
plt.plot(histImage)
plt.plot(histMI)

---  


# 直方图均衡化
如果一幅图像拥有全部可能的灰度级，并且像素值的灰度均匀分布，那么这幅图像就具有高对比度和多变的灰度色调，灰度级丰富且覆盖范围较大。在外观上，这样的图像具有更丰富的色彩，不会过暗或过亮。

直方图均衡化的主要目的是将原始图像的灰度级均匀地映射到整个灰度级范围内，得到一个灰度级分布均匀的图像。这种均衡化，既实现了灰度值统计上的概率均衡，也实现了人类视觉系统（Human Visual System, HVS）上的视觉均衡。

## 直方图均衡化原理
直方图均衡化的算法主要包括两个步骤：
- （1）计算累计直方图。
- （2）对累计直方图进行区间转换

### 在原有范围内实现均衡化
在原有范围内实现直方图均衡化时，用当前灰度级的累计概率乘以当前灰度级的最大值7，得到新的灰度级，并作为均衡化的结果。

### 在更广泛的范围内实现均衡化
在更广泛的范围内实现直方图均衡化时，用当前灰度级的累计概率乘以更广泛范围灰度级的最大值，得到新的灰度级，并作为均衡化的结果。

## 直方图均衡化处理
OpenCV使用函数`cv2.equalizeHist()`实现直方图均衡化。

In [ ]:
#-----------读取原始图像---------------
img = cv2.imread('./images/equ.bmp',cv2.IMREAD_GRAYSCALE)
#-----------直方图均衡化处理---------------
equ = cv2.equalizeHist(img)
#-----------显示均衡化前后的直方图---------------
cv2.imshow("original",img)
cv2.imshow("result",equ)
#-----------显示均衡化前后的直方图---------------
plt.figure("原始图像直方图")  #构建窗口
plt.hist(img.ravel(),256)
plt.figure("均衡化结果直方图")  #构建新窗口
plt.hist(equ.ravel(),256)
#----------等待释放窗口---------------------
cv2.waitKey()
cv2.destroyAllWindows()

均衡化是指综合考虑了统计概率和HVS的均衡化结果。

---

# pyplot模块介绍
## subplot函数
模块matplotlib.pyplot提供了函数`matplotlib.pyplot.subplot()`用来向当前窗口内添加一个子窗口对象。


In [ ]:
img = cv2.imread('./images/equ.bmp',cv2.IMREAD_GRAYSCALE)
equ = cv2.equalizeHist(img)
plt.figure("subplot示例")
plt.subplot(121),plt.hist(img.ravel(),256)
plt.subplot(122),plt.hist(equ.ravel(),256)

### imshow函数
模块matplotlib.pyplot提供了函数`matplotlib.pyplot.imshow()`用来显示图像。

In [ ]:
img = cv2.imread('images/girl.bmp')
imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.figure("显示结果")
plt.subplot(121)
plt.imshow(img),plt.axis('off')
plt.subplot(122)
plt.imshow(imgRGB),plt.axis('off')

In [ ]:
o = cv2.imread('images/girl.bmp')
g=cv2.cvtColor(o, cv2.COLOR_BGR2GRAY)
plt.figure("灰度图像显示演示")
plt.subplot(221)
plt.imshow(o),plt.axis('off')
plt.subplot(222)
plt.imshow(o,cmap=plt.cm.gray),plt.axis('off')
plt.subplot(223)
plt.imshow(g),plt.axis('off')
plt.subplot(224)
plt.imshow(g,cmap=plt.cm.gray),plt.axis('off')

In [ ]:
o = cv2.imread('images/8.bmp')
g=cv2.cvtColor(o, cv2.COLOR_BGR2GRAY)
plt.figure("灰度图像显示演示")
plt.subplot(221); plt.imshow(g, cmap=plt.cm.gray)
plt.subplot(222); plt.imshow(g, cmap=plt.cm.gray_r)
plt.subplot(223); plt.imshow(g, cmap='gray')
plt.subplot(224); plt.imshow(g, cmap='gray_r')

---